In [1]:
!git clone https://github.com/parkerburchett/pysmiles
!pip install pyTDC
!git clone https://github.com/parkerburchett/TDC-DeepLearning

Cloning into 'pysmiles'...
remote: Enumerating objects: 420, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 420 (delta 12), reused 18 (delta 4), pack-reused 390
Receiving objects: 100% (420/420), 134.29 KiB | 5.17 MiB/s, done.
Resolving deltas: 100% (251/251), done.
     |████████████████████████████████| 90 kB 2.7 MB/s 
  Created wheel for pyTDC: filename=PyTDC-0.3.2-py3-none-any.whl size=117596 sha256=d627a2f159363f2d2fb93b3821f84b09428b5f7415f1faf456ac6f496b47da0a
  Stored in directory: /root/.cache/pip/wheels/fe/ef/02/99f35109c3a026965faad8b3cf0f8aeb3a9e01e069f40f8698
Successfully built pyTDC
Cloning into 'TDC-DeepLearning'...
remote: Enumerating objects: 268, done.
remote: Counting objects: 100% (268/268), done.
remote: Compressing objects: 100% (209/209), done.
remote: Total 268 (delta 97), reused 224 (delta 54), pack-reused 0
Receiving objects: 100% (268/268), 49.21 MiB | 17.72 MiB/s, done.
Resolving deltas: 100

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import lightgbm as lgb
import json
from sklearn.metrics import precision_recall_curve, auc
from pysmiles.pysmiles import read_smiles
import os
os.chdir('/content/TDC-DeepLearning/')
from utils import ColorRefinement as cr 
from tdc.benchmark_group import admet_group

Mounted at /content/drive


In [4]:
best_models = json.load(open('/content/drive/MyDrive/SpringBoard/Therapeutic Data Commons Projects/HyperParamTuning/bestmodels.json', 'r'))
best_models

{'0': "{'colsample_bytree': 0.45921506474872353, 'learning_rate': 0.003605978989205916, 'n_estimators': 997, 'num_leaves': 171, 'reg_alpha': 0.06136193030050688, 'subsample': 0.664374000848817}",
 '1': "{'colsample_bytree': 0.19190373976042552, 'learning_rate': 0.018880733945270692, 'n_estimators': 772, 'num_leaves': 412, 'reg_alpha': 0.1319602189105627, 'subsample': 0.953435263598222}",
 '2': "{'colsample_bytree': 0.0846115062976256, 'learning_rate': 0.061904626017968235, 'n_estimators': 710, 'num_leaves': 218, 'reg_alpha': 0.09722107305351997, 'subsample': 0.7625401046034898}",
 '3': "{'colsample_bytree': 0.10234165146414909, 'learning_rate': 0.021876318417714605, 'n_estimators': 574, 'num_leaves': 193, 'reg_alpha': 0.08093256266597965, 'subsample': 0.9986613307559011}",
 '4': "{'colsample_bytree': 0.29036206035748535, 'learning_rate': 0.09316958191707549, 'n_estimators': 473, 'num_leaves': 101, 'reg_alpha': 0.1992406954388929, 'subsample': 0.005705227464680718}"}

In [5]:
# load data
import glob
import lightgbm as lgb
def get_larger_file_names():
  """
    Returns the file locations for the 10_000 bucket color embeddings.
  """
  return glob.glob('/content/drive/MyDrive/SpringBoard/Therapeutic Data Commons Projects/data/more_buckets_data/*hop_larger_embedding.csv')

def load_feature_df(hop_num:int):
  """
      Load all the color features, target of hop number hop nu
      Return a dataframe of (color0, color1 ... color998,color999, target) of the insample training data
  """
  hop_file_name = get_larger_file_names()[hop_num]
  df = pd.read_csv(hop_file_name, index_col=0)
  df = df.astype(np.int16)
  group = admet_group(path = 'data/')
  benchmark = group.get('cyp2c9_veith')
  targets =  benchmark['train_val']['Y'].astype(np.bool)
  df['target'] =  targets
  return df

from sklearn.metrics import precision_recall_curve, auc

def compute_auprc(y_true, y_pred):
    # https://stats.stackexchange.com/questions/157012/area-under-precision-recall-curve-auc-of-pr-curve-and-average-precision-ap
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    area = round(auc(recall, precision), 6)
    return area

In [6]:
def create_model_df_set(hop_num, params):
  params_as_dict = json.loads(params.replace("'", '"'))
  return {'hop_number' : hop_num,
          'model_object': lgb.LGBMRegressor(**params_as_dict, subsample_freq=1),
          'df' : load_feature_df(hop_num)}

model_df_pairs = [create_model_df_set(i, best_models[str(i)]) for i in range(0,4)]


Found local copy...
Found local copy...
Found local copy...
Found local copy...


In [10]:
for d in model_df_pairs:

  d['key'] = d.pop('hop_number')
  d['training_df'] = d.pop('df')

KeyError: ignored

In [11]:
import pickle 

pickle.dump(model_df_pairs, open('/content/drive/MyDrive/SpringBoard/Therapeutic Data Commons Projects/HyperParamTuning/model_df_pairs.pkl', 'wb'))

In [9]:
# from sklearn.model_selection import KFold

# df = model_df_pairs[0]['df']
# features = [f for f in df.columns if 'color' in f]

# kf = KFold(n_splits=5, random_state=None, shuffle=False)

# X = df[features].values

# prediction_dfs = []
# # gets 25 predictions
# for train_index, test_index in kf.split(X): # only used to get the indexes correct
#   prediction_df = pd.DataFrame(index=test_index)

#   for pairs in model_df_pairs:
#     df = pairs['df']
#     hop_num = pairs['hop_number']
#     model = pairs['model_object']

#     X = df[features].values
#     y = df['target'].values

#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

#     model.fit(X_train, y_train)
#     prediction_df[f'model_{hop_num}'] = model.predict(X_test)

#     if 'target' not in prediction_df.columns:
#       prediction_df['target'] = y_test

#   print(prediction_df.head())
#   prediction_dfs.append(prediction_df)




KeyError: ignored

In [16]:
model_df_pairs[0]

{'key': 0,
 'model_object': LGBMRegressor(boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.45921506474872353, importance_type='split',
               learning_rate=0.003605978989205916, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=997, n_jobs=-1, num_leaves=171, objective=None,
               random_state=None, reg_alpha=0.06136193030050688, reg_lambda=0.0,
               silent=True, subsample=0.664374000848817,
               subsample_for_bin=200000, subsample_freq=1),
 'training_df':        color_0  color_1  color_2  ...  color_9998  color_9999  target
 index                             ...                                
 0            0        0        0  ...           0           0   False
 1            0        0        0  ...           0           0   False
 2            0        0        0  ...           0           0   False
 3            0        0        0  ...           

In [24]:

df = model_df_pairs[0]['training_df']
features = [f for f in df.columns if 'color' in f]

def create_model_feature_pairs(model_objects:list, training_dfs:list) -> dict:
  """
    Create a list of dictionaries of
    {
      key: an int id,
      model_object: a model object
      feature_df: a dataframe of features, and target to train model_object on
    } 

    This is a component that is used to create a weighted classifier.
    You can use this later on the full data to create the component for the prediction
  """
  if len(model_objects) != len(training_dfs):
    raise ValueError('you must pass in the same number of model and training dfs')
  
  model_feature_pairs = []
  for key in range(len(model_objects)):
    pair = {
        'key':key,
        'model_object': model_objects[key],
        'training_df': training_dfs[key]
    }
    model_feature_pairs.append(pair)
  
  return model_feature_pairs

def create_prediction_dfs_for_weighted_classifier(model_feature_pairs:list,
                                                  k_fold_n_splits:int,
                                                  k_fold_shuffle:bool,
                                                  target_col:str) -> list:
  """

    Create a list of dfs with shape
    (len(model_feature_pairs) + 1, rows in a feature df /  k_fold_n_splits


    [      
            model_0  target   model_1   model_2   model_3
    7739  0.255534   False  0.371322  0.581279  0.297537
    7740 -0.022156   False -0.002481 -0.068137 -0.082516
    7741  0.598179    True  0.423898  0.541822  0.675032
    7742  0.208997   False  0.190605  0.110511  0.280596
    7743  0.050421   False  0.128555  0.369134  0.347044
    ,
           model_0  target   model_1   model_2   model_3
    1935  0.312784   False  0.208339  0.295249  0.286069
    1936  0.334130    True  0.347660  0.449999  0.583438
    1937  0.460933    True  0.445365  0.691044  0.297753
    1938  0.426971    True  0.588611  0.686233  0.507866
    1939  0.859865    True  0.984376  1.068336  0.679388
    ]


    You use this to create a weighted classifier later
  """

  kf = KFold(n_splits=k_fold_n_splits, random_state=None, shuffle=k_fold_shuffle)

  # this is ugly but is needed to do the Kfold splits
  df = model_df_pairs[0]['training_df']
  X = df[features].values
  prediction_dfs = []

  for train_index, test_index in kf.split(X): # only used to get the indexes correct
    prediction_df = pd.DataFrame(index=test_index)

    for pair in model_df_pairs:
      
      
      key = pair['key']
      training_df = pair['training_df']
      model = pair['model_object']
      feature_cols = [f for f in training_df.columns if f!=target_col]

      X = training_df[feature_cols].values
      y = training_df[target_col].values

      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]

      model.fit(X_train, y_train)
      prediction_df[f'model_{key}'] = model.predict(X_test)

      if target_col not in prediction_df.columns:
        prediction_df[target_col] = y_test

    print(prediction_df.head())
    prediction_dfs.append(prediction_df)
    
  return prediction_dfs




prediction_dfs = create_prediction_dfs_for_weighted_classifier(
                model_feature_pairs=model_df_pairs,
                k_fold_n_splits=5,
                k_fold_shuffle=False,
                target_col='target'
                )


    model_0  target   model_1   model_2   model_3
0  0.148717   False  0.168826  0.126168  0.326254
1  0.147821   False  0.009772  0.011469  0.058126
2  0.109165   False -0.164792  0.358199  0.199572
3  0.147362   False  0.024964  0.317119  0.370499
4  0.132685   False  0.315626  0.347921  0.351123
       model_0  target   model_1   model_2   model_3
1935  0.312784   False  0.208339  0.295249  0.286069
1936  0.334130    True  0.347660  0.449999  0.583438
1937  0.460933    True  0.445365  0.691044  0.297753
1938  0.426971    True  0.588611  0.686233  0.507866
1939  0.859865    True  0.984376  1.068336  0.679388
       model_0  target   model_1   model_2   model_3
3870  0.459636   False  0.447652  0.706204  0.545787
3871  0.329500    True  0.449534  0.352346  0.194716
3872  0.008223   False  0.123109 -0.060045  0.231775
3873  0.577824   False  0.351500  0.414599  0.285807
3874  0.687691    True  0.864578  0.972968  0.991008
       model_0  target   model_1   model_2   model_3
5805  0.261

In [25]:
prediction_dfs[3] # example prediction on the test set at hop 3. Note this does not not look at the out of sample validation data.s


,model_0,target,model_1,model_2,model_3
5805,0.261044,False,0.142227,0.157438,0.161204
5806,0.533139,True,0.339514,0.497814,0.170840
5807,0.392724,False,0.311528,0.424494,0.599304
5808,0.183220,False,0.352198,0.642930,0.678630
5809,0.387226,False,0.106226,-0.038921,0.063332
...,...,...,...,...,...
7734,0.811472,True,1.025497,1.038222,0.905511
7735,0.755061,True,0.714088,0.533547,0.377616
7736,0.375517,False,0.383989,0.519285,0.495733
7737,0.063435,False,0.007947,-0.039301,-0.004630


In [26]:
def create_random_weight_array(n=4):
  percent_weights = np.random.randint(0,100,n) # note this does not randomly distribute them. It is not necessarily even
  percent_weights = percent_weights / percent_weights.sum()
  return percent_weights

In [27]:
N = 100_000 # 100K takes 15 minutes
outcome_lists = np.zeros((N, 5))
model_cols = [c for c in prediction_dfs[0].columns if 'model' in c]
for i in range(N):
  weights = create_random_weight_array()
  auprc_scores = np.zeros(5)
  for split_num, split_df in enumerate(prediction_dfs):
    
    weighted_combination = split_df[model_cols].values.dot(weights)
    auprc_scores[split_num] = compute_auprc(split_df['target'], weighted_combination)

  outcome_lists[i]  =  [*weights, np.mean(auprc_scores)]

In [29]:
weight_ensample_df = pd.DataFrame(outcome_lists)
weight_ensample_df.columns = ['weight_1', 'weight_2', 'weight_3', 'weight_4', 'auprc']
weight_ensample_df = weight_ensample_df.sort_values('auprc', ascending=False)

In [30]:
weight_ensample_df.head(20)

,weight_1,weight_2,weight_3,weight_4,auprc
31079,0.228070,0.508772,0.157895,0.105263,0.796028
4834,0.232394,0.507042,0.183099,0.077465,0.796014
93031,0.237113,0.505155,0.159794,0.097938,0.796001
62021,0.236994,0.502890,0.179191,0.080925,0.795998
11659,0.236842,0.506579,0.157895,0.098684,0.795997
93146,0.246667,0.493333,0.166667,0.093333,0.795983
85659,0.228814,0.500000,0.169492,0.101695,0.795982
36697,0.238095,0.510204,0.163265,0.088435,0.795980
97045,0.238636,0.505682,0.176136,0.079545,0.795968
28368,0.231707,0.524390,0.158537,0.085366,0.795968


In [31]:
print(weight_ensample_df.head(20).mean())

print(weight_ensample_df.head(20).std())

weight_1    0.232034
weight_2    0.508359
weight_3    0.169162
weight_4    0.090446
auprc       0.795955
dtype: float64
weight_1    0.009602
weight_2    0.015332
weight_3    0.016236
weight_4    0.013102
auprc       0.000042
dtype: float64


In [32]:
best_weights =  weight_ensample_df.head(20).mean()
print(best_weights)
best_weights.to_csv('/content/drive/MyDrive/SpringBoard/Therapeutic Data Commons Projects/HyperParamTuning/best_4_regression_weights.csv')

weight_1    0.232034
weight_2    0.508359
weight_3    0.169162
weight_4    0.090446
auprc       0.795955
dtype: float64
